### DART 사업보고서(최종) 목록

* 사업보고서가 아닌 내역을 삭제한다
* 사업보고서 접수년도별로 구분하여 파일로 저장

In [18]:
import os
import pandas as pd

import sys
module_path = "D:/PythonProject/data-gatherer/dart_fs_notes"
sys.path.append(module_path)
import mydart

In [19]:
# DART 보고서 파일 목록 위치
path_dir_rpt = os.path.join(mydart.root_path, "dart_rpt")
rpt_file_list = os.listdir(path_dir_rpt)

# DART 보고서 파일들의 연도
years = []
for file_name in rpt_file_list:
    year = file_name[2:6]
    years.append(year)
    
years = list(set(years))    # 중복 제거

# 사업보고서 파일 저장 위치
path_dir_bizrpt = os.path.join(mydart.root_path, "dart_bizrpt")
mydart.createDirectory(path_dir_bizrpt)

for y in years:
    # 해당 연도에 해당하는 모든 파일을 읽어서 보고서 목록을 하나로 합한다.
    year_file_list = filter(lambda x: x[2:6] == y, rpt_file_list)
    df_all = pd.DataFrame()    
    for file_name in year_file_list:        
        if file_name.startswith('A_') and not file_name.startswith('__'):
            file_path = os.path.join(path_dir_rpt, file_name)
            df = pd.read_csv(file_path, dtype=object)
            df_all = pd.concat([df_all, df])        
    # "사업보고서"가 아닌 것들 삭제. "사업보고서"만 목록에 남긴다.
    df_biz_rpt = df_all.loc[(df_all.report_nm.str.contains('사업보고서')) & 
                            (~df_all.report_nm.str.contains('신고서')) &
                            (~df_all.report_nm.str.contains('외국기업의경우'))
                            , ]
    df_biz_rpt.reset_index()
    # 파일로 저장
    bizrpt_file_path = os.path.join(path_dir_bizrpt, "bizrpt_" + y + ".csv")
    if not os.path.isfile(bizrpt_file_path):   # 파일 존재 유무 체크
        df_biz_rpt.to_csv(bizrpt_file_path, index=False)

* 사업보고서 명칭 오류 --> 수기 보정

In [21]:
# DART 사업보고서 파일 목록 위치
path_dir_rpt = os.path.join(mydart.root_path, "dart_bizrpt")
bizrpt_file_list = os.listdir(path_dir_rpt)

# DART 사업보고서 접수연도별 Dataframe
bizrpt_years = []
for file_name in bizrpt_file_list:
    if not file_name.startswith('bizrpt_all') and not file_name.startswith('__'):
        year = file_name[7:11]
        file_path = os.path.join(path_dir_rpt, file_name)
        df_bizrpt = pd.read_csv(file_path, dtype=object)
        bizrpt_years.append({'year' : year, 'file' : file_name, 'data' : df_bizrpt})

# 수기보정 필요한 사항 있는지 체크
for r in bizrpt_years:
    df = r['data']
    # report_nm 명칭이 (결산년월)로 끝나지 않은 것
    df_abnormal = df.loc[~df.report_nm.str.endswith(')'), ]
    if df_abnormal.shape[0] > 0:
        print(r['year'], r['file'], df_abnormal.index)

2022 bizrpt_2022.csv Index([2728], dtype='int64')


In [22]:
file_name = "bizrpt_2022.csv"
file_path = os.path.join(path_dir_rpt, file_name)
df_bizrpt_2022 = pd.read_csv(file_path, dtype=object)
# 수기보정이 필요한 데이터(index 번호로 검색)
df_bizrpt_2022.iloc[2728, ]

corp_code           00118521
corp_name             진원생명과학
stock_code            011000
corp_cls                   Y
report_nm        [기재정정]사업보고서
rcept_no      20220908000421
flr_nm                진원생명과학
rcept_dt            20220908
rm                       NaN
Name: 2728, dtype: object

In [23]:
# DART에서 직접 확인한 내용으로 수정
# corp_name '진원생명과학', rcept_no '20220908000421'
df_bizrpt_2022.loc[2728,'report_nm'] = "[기재정정]사업보고서 (2005.12)"
# 파일 재저장
file_name = "bizrpt_2022.csv"
file_path = os.path.join(path_dir_rpt, file_name)
df_bizrpt_2022.to_csv(file_path, index=False)

* 이상치 데이터 수기보정 후 다시 체크

In [24]:
# DART 사업보고서 파일 목록 위치
path_dir_rpt = os.path.join(mydart.root_path, "dart_bizrpt")
bizrpt_file_list = os.listdir(path_dir_rpt)

# DART 사업보고서 접수연도별 Dataframe
bizrpt_years = []
for file_name in bizrpt_file_list:
    if not file_name.startswith('bizrpt_all') and not file_name.startswith('__'):
        year = file_name[7:11]
        file_path = os.path.join(path_dir_rpt, file_name)
        df_bizrpt = pd.read_csv(file_path, dtype=object)
        bizrpt_years.append({'year' : year, 'file' : file_name, 'data' : df_bizrpt})

# 수기보정 필요한 사항 있는지 체크
for r in bizrpt_years:
    df = r['data']
    # report_nm 명칭이 (결산년월)로 끝나지 않은 것
    df_abnormal = df.loc[~df.report_nm.str.endswith(')'), ]
    if df_abnormal.shape[0] > 0:
        print(r['year'], r['file'], df_abnormal.index)